In [1]:
from PIL import Image
import os

In [2]:
dir = "/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/"

In [3]:
# for filename in os.listdir(dir):
#     if "._" not in filename and ".DS" not in filename:
         
#         image = Image.open(dir + filename)
#         image2 = image.crop((300, 600, 1024, 1500))
#         image3 = image.crop((1024, 600, image.size[0] - 300, 1500))
#         image2.save("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/test/other/" + filename)
#         image3.save("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/test/other/2" + filename)

In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

In [3]:
vgg16 = models.vgg16_bn()
vgg16.load_state_dict(torch.load("/home/jszumny/Downloads/vgg16_bn-6c64b313.pth"))
print(vgg16.classifier[6].out_features) # 1000

1000


In [6]:
resume_training = True

if resume_training:
    
    print("Loading pretrained model..")
    vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/fullmodelbest.pt',map_location=torch.device('cpu')))
    print("Loaded!")

Loading pretrained model..


RuntimeError: Error(s) in loading state_dict for VGG:
	size mismatch for classifier.6.weight: copying a param with shape torch.Size([3, 4096]) from checkpoint, the shape in current model is torch.Size([1000, 4096]).
	size mismatch for classifier.6.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([1000]).

In [6]:
for param in vgg16.features.parameters():
    param.require_grad = False
class_names = ['cloud', 'other', 'smoke']

In [7]:
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

In [8]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])



# image = Image.open("/home/jszumny/Downloads/smokee.jpeg")
# img_t = transform(image)
# batch_t = torch.unsqueeze(img_t, 0)
# image.show()


In [ ]:
vgg16.eval()
out = vgg16(batch_t)
print(out.shape)


torch.Size([1, 3])


In [ ]:
_, index = torch.max(out, 1)
 
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
 
print(class_names[index[0]], percentage[index[0]].item())

cloud 42.42892837524414


In [ ]:
_, indices = torch.sort(out, descending=True)
[(class_names[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('cloud', 42.42892837524414),
 ('other', 30.903717041015625),
 ('smoke', 26.6673526763916)]

In [12]:
list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clear_sky/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/fullmodelbest.pt',map_location=torch.device('cpu')))

        for param in vgg16.features.parameters():
            param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clear_sky/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        vgg16.eval()
        out = vgg16(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)


In [13]:
len(df[df["Class"] == "smoke"]) / len(df)

0.008403361344537815

In [14]:
df

,Class,Percentage
0,other,50.678963
1,other,50.200634
2,cloud,94.633087
3,cloud,56.009666
4,cloud,42.246296
...,...,...
114,cloud,57.728077
115,cloud,49.652565
116,other,41.612331
117,other,43.622978


In [ ]:
list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clouds/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/fullmodelbest.pt',map_location=torch.device('cpu')))

        for param in vgg16.features.parameters():
            param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clouds/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        vgg16.eval()
        out = vgg16(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df2 = pd.DataFrame(list)

In [ ]:
df2

,Class,Percentage
0,cloud,88.528069
1,smoke,44.058887
2,cloud,99.323601
3,smoke,84.997543
4,cloud,61.817337
...,...,...
775,other,92.928329
776,cloud,99.516312
777,cloud,99.628181
778,cloud,71.874435


In [ ]:
len(df2[df2["Class"] == "smoke"]) / len(df2)

0.05

In [ ]:
list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/fullmodelbest.pt',map_location=torch.device('cpu')))

        for param in vgg16.features.parameters():
            param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        vgg16.eval()
        out = vgg16(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df3 = pd.DataFrame(list)

In [ ]:
df3

,Class,Percentage
0,other,79.876564
1,cloud,77.192413
2,other,49.888454
3,other,89.901848
4,other,84.399361
...,...,...
451,other,97.816055
452,other,72.938225
453,cloud,63.851238
454,cloud,86.317787


In [ ]:
len(df3[df3["Class"] == "smoke"]) / len(df3)


0.019736842105263157

UCSD MODEL

In [9]:
# from __future__ import print_function, division

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.optim import lr_scheduler
# from torch.autograd import Variable
# import numpy as np
# import torchvision
# from torchvision import datasets, models, transforms
# import matplotlib.pyplot as plt
# import time
# import os
# import copy
# # from main_model import MainModel

# plt.ion()

# use_gpu = torch.cuda.is_available()
# if use_gpu:
#     print("Using CUDA")

In [9]:

resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# resnet34 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
num_features = resnet18.fc.in_features     #extract fc layers features
resnet18.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

# num_features = resnet34.fc.in_features     #extract fc layers features
# resnet34.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

# num_features = resnet50.fc.in_features     #extract fc layers features
# resnet50.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clear_sky/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet18.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet18model.pt',map_location=torch.device('cpu')))

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clear_sky/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet18.eval()
        out = resnet18(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
df[df.Class == "smoke"]

,Class,Percentage


In [ ]:
list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet18.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet18model.pt',map_location=torch.device('cpu')))

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet18.eval()
        out = resnet18(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

In [ ]:
len(df[df.Class == "smoke"]) / len(df
        )

0.021929824561403508

In [12]:
list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clouds/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet18.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet18model.pt',map_location=torch.device('cpu')))

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clouds/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet18.eval()
        out = resnet18(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

KeyboardInterrupt: 

In [ ]:
len(df[df.Class == "smoke"]) / len(df)


0.008974358974358974

In [29]:
# resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
resnet34 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# num_features = resnet18.fc.in_features     #extract fc layers features
# resnet18.fc = nn.Linear(num_features, 3) #(num_of_class == 2)


num_features = resnet34.fc.in_features     #extract fc layers features
resnet34.fc = nn.Linear(num_features, 3) #(num_of_class == 2)


# num_features = resnet50.fc.in_features     #extract fc layers features
# resnet50.fc = nn.Linear(num_features, 3) #(num_of_class == 2)



list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clear_sky/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet34.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet34model.pt',map_location=torch.device('cpu')))
        

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clear_sky/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet34.eval()
        out = resnet34(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [30]:
df[df.Class == "smoke"]

,Class,Percentage


In [31]:
list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clouds/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet34.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet34model.pt',map_location=torch.device('cpu')))
        

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clouds/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet34.eval()
        out = resnet34(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

In [32]:
len(df[df.Class == "smoke"]) / len(df)

0.046153846153846156

In [25]:
list = []
for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet34.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet34model.pt',map_location=torch.device('cpu')))
        

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet34.eval()
        out = resnet34(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

In [28]:
len(df[df.Class == "smoke"]) / len(df)

0.021929824561403508

In [34]:
list = []


resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

num_features = resnet50.fc.in_features     #extract fc layers features
resnet50.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet50.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet50model.pt',map_location=torch.device('cpu')))
        

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet50.eval()
        out = resnet50(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0


In [37]:
len(df[df.Class == "smoke"])/ len(df)

0.05921052631578947

In [38]:
list = []


resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

num_features = resnet50.fc.in_features     #extract fc layers features
resnet50.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clear_sky/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet50.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet50model.pt',map_location=torch.device('cpu')))
        

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clear_sky/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet50.eval()
        out = resnet50(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
df[df.Class == "smoke"]

,Class,Percentage


In [41]:
list = []


resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

num_features = resnet50.fc.in_features     #extract fc layers features
resnet50.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

for filename in os.listdir("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clouds/"):

    if "._" not in filename and ".DS_Store" not in filename:
        
        resnet50.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/resnet50model.pt',map_location=torch.device('cpu')))
        

        # for param in resnet18.features.parameters():
        #     param.require_grad = False
        class_names = ['cloud', 'other', 'smoke']

        # num_features = vgg16.classifier[6].in_features
        # features = list(vgg16.classifier.children())[:-1] # Remove last layer
        # features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
        # vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        image = Image.open("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/clouds/" + filename)
        image = image.crop((874, 874, 1174, 1174))
        img_t = transform(image)
        batch_t = torch.unsqueeze(img_t, 0)
    
        resnet50.eval()
        out = resnet50(batch_t)
    
        _, index = torch.max(out, 1)
    
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        
        d = {"Class": class_names[index[0]], "Percentage": percentage[index[0]].item()}
        list.append(d)

import pandas as pd
df = pd.DataFrame(list)

Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
len(df[df.Class == "smoke"]) / len(df)

0.019230769230769232